In [1]:
import sys
sys.path.append('../../Share/')
sys.path.append('../../V5 Self Supervised Learning')

import pandas as pd
import numpy as np
from sklearn.utils import resample
import matplotlib.pyplot as plt
import baseline, config, self_supervised_v1

import warnings
warnings.filterwarnings('ignore')

def evaluate_model(model, data, labels):
    return model.evaluate(data, labels, verbose=0)[1]


def down_sample(X, y):
    X = np.array(X)
    y = np.array(y)

    # 클래스별 최소 개수
    unique_classes, counts = np.unique(y, return_counts=True)
    min_count = counts.min()

    X_balanced, y_balanced = [], []

    for cls in unique_classes:
        idx = np.where(y == cls)[0]  # 해당 클래스 인덱스
        down_idx = resample(idx, replace=False, n_samples=min_count, random_state=42)
        X_balanced.append(X[down_idx])
        y_balanced.append(y[down_idx])

    # 합치기
    X_balanced = np.concatenate(X_balanced, axis=0)
    y_balanced = np.concatenate(y_balanced, axis=0)

    return X_balanced, y_balanced



trainer_Minjeong = baseline.ModelTrainer(config, subject="Minjeong")
trainer_Carlson = baseline.ModelTrainer(config, subject="Carlson")
trainer_Harold = baseline.ModelTrainer(config, subject="Harold")
trainer_Hunmin = baseline.ModelTrainer(config, subject="Hunmin")
trainer_Brian = baseline.ModelTrainer(config, subject="Brian")
trainer_Xianyu = baseline.ModelTrainer(config, subject="Xianyu")

In [2]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

model = load_model('./model_K10_H2.h5')

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

trainers_unseen = {
    "H": trainer_Hunmin,
    "C": trainer_Carlson,
    "B": trainer_Brian,
    "X": trainer_Xianyu,
    "M": trainer_Minjeong
}
#Same_Session_Test_Acc, Next_Session_Test_Acc = [], []
#Unseen_subject_acc_dict = {"H": [], "C": [], "B": [], "X": [], "M": []}

In [ ]:
import meta

trainer_sub = trainer_Carlson
final_session = len(config.Info_sub_C)
X_TEST, y_TEST, _, _ = trainer_sub.return_K_th_data_only(K=final_session-1, train_ratio=0.99)

Acc_lst = []

for k in range(final_session-1):
    unseen_X, unseen_y, X_few_shot, y_few_shot = trainer_sub.return_K_th_data_only(K=k, train_ratio=0.8)
    X_train, y_train = down_sample(unseen_X, unseen_y)
    X_test, y_test = down_sample(X_few_shot, y_few_shot)

    Meta = meta.MetaLearner(input_model=model, N_way=6, input_shape=X_train.shape[1:],
        meta_iters=5,                              # Number of meta-training loops
        meta_step_size=0.2                          # Reptile meta step
    )

    Meta.train(X_train, y_train, X_test, y_test, meta.get_data_Meta, N_way=6, K_shot=20)
    acc = evaluate_model(model, X_TEST, y_TEST)
    print(acc)
    Acc_lst.append(acc)

Returning K-th session data: Exp_2025-08-01-v2/E9AD0E7DCC2B/
Returning K-th session data: Exp_2025-06-30-v1/E9AD0E7DCC2B/
0.8416273593902588
Returning K-th session data: Exp_2025-06-30-v2/E9AD0E7DCC2B/
